In [28]:
from Tokenizer import *

path_to_curpus = 'sample.txt'

#convert the corpus to a list of tokens
sentences ,freq ,tokens = tokenize(path_to_curpus )
print("ruk jaa bhia ")


ruk jaa bhia 


In [29]:
import torch
# split into train , test and validation sets with random shuffling
train_size = 0.8
test_size = 0.1
validation_size = 0.1
print(len(sentences))
#remove 6 senteces from the end of the list
# sentences = sentences[:-6]
# prit
proportions = [.75, .10, .15]
lengths = [int(p * len(sentences)) for p in proportions]
lengths[-1] = len(sentences) - sum(lengths[:-1])
train , test , validation = torch.utils.data.random_split(sentences, lengths)
# train , test , validation = torch.utils.data.random_split(sentences , [int(train_size*len(sentences)) , int(test_size*len(sentences)) , int(validation_size*len(sentences))])

print(train , test , validation)

8
<torch.utils.data.dataset.Subset object at 0x7f8cf7fc94b0> <torch.utils.data.dataset.Subset object at 0x7f8cf7fc8970> <torch.utils.data.dataset.Subset object at 0x7f8cf81451b0>


In [30]:
#give unique index to each token and create a vocabulary
# vocab = {token:idx for idx,token in enumerate(tokens)}
vocab = {"<pad>": 0}  # Add a <pad> token to the vocab
inputs = []
max_length = 0  # Track the maximum sequence length


for sentence in train :
    sentence = "<s> "*3 +sentence + " </s>"
    sentence_indexes = []
    words = sentence.split()
    for word in words:
        if word not in vocab:
            vocab[word] = len(vocab)
        sentence_indexes.append(vocab[word])
    inputs.append(sentence_indexes)
    max_length = max(max_length, len(sentence_indexes))
    
print("Max sequence length:", max_length)
# print("Shape of inputs before padding:", torch.LongTensor(inputs).shape)

# print(vocab)
print(inputs)

Max sequence length: 11
[[1, 1, 1, 2, 3, 4, 5, 6, 7], [1, 1, 1, 8, 9, 4, 5, 6, 10, 11, 7], [1, 1, 1, 12, 9, 13, 7], [1, 1, 1, 7], [1, 1, 1, 2, 3, 4, 5, 14, 7], [1, 1, 1, 12, 9, 4, 5, 6, 10, 14, 7]]


In [31]:
# Pad the input sequences with the <pad> token to make them all the same length
for i in range(len(inputs)):
    inputs[i] = inputs[i] + [vocab["<pad>"]] * (max_length - len(inputs[i]))


print(inputs)


[[1, 1, 1, 2, 3, 4, 5, 6, 7, 0, 0], [1, 1, 1, 8, 9, 4, 5, 6, 10, 11, 7], [1, 1, 1, 12, 9, 13, 7, 0, 0, 0, 0], [1, 1, 1, 7, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 2, 3, 4, 5, 14, 7, 0, 0], [1, 1, 1, 12, 9, 4, 5, 6, 10, 14, 7]]


In [32]:
import torch 
#convert to LongTensor
prefixes = []
labels = []
for i in range(len(inputs)):
    for j in range(len(inputs[i])-3):
        if(inputs[i][j+3]==0): 
           break          
        prefix = inputs[i][j:j+3]
        label = inputs[i][j+3]
        # print(prefix , label)
        prefixes.append(prefix)
        labels.append(label)
    #3-gram prefix


    
prefixes  = torch.LongTensor([sentence[:-1] for sentence in inputs])
labels = torch.LongTensor([sentence[1:] for sentence in inputs])


print(prefixes , labels)

#Define inputs and outputs 

[1, 1, 1] 2
[1, 1, 2] 3
[1, 2, 3] 4
[2, 3, 4] 5
[3, 4, 5] 6
[4, 5, 6] 7
[1, 1, 1] 8
[1, 1, 8] 9
[1, 8, 9] 4
[8, 9, 4] 5
[9, 4, 5] 6
[4, 5, 6] 10
[5, 6, 10] 11
[6, 10, 11] 7
[1, 1, 1] 12
[1, 1, 12] 9
[1, 12, 9] 13
[12, 9, 13] 7
[1, 1, 1] 7
[1, 1, 1] 2
[1, 1, 2] 3
[1, 2, 3] 4
[2, 3, 4] 5
[3, 4, 5] 14
[4, 5, 14] 7
[1, 1, 1] 12
[1, 1, 12] 9
[1, 12, 9] 4
[12, 9, 4] 5
[9, 4, 5] 6
[4, 5, 6] 10
[5, 6, 10] 14
[6, 10, 14] 7
tensor([[ 1,  1,  1,  2,  3,  4,  5,  6,  7,  0],
        [ 1,  1,  1,  8,  9,  4,  5,  6, 10, 11],
        [ 1,  1,  1, 12,  9, 13,  7,  0,  0,  0],
        [ 1,  1,  1,  7,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  1,  2,  3,  4,  5, 14,  7,  0],
        [ 1,  1,  1, 12,  9,  4,  5,  6, 10, 14]]) tensor([[ 1,  1,  2,  3,  4,  5,  6,  7,  0,  0],
        [ 1,  1,  8,  9,  4,  5,  6, 10, 11,  7],
        [ 1,  1, 12,  9, 13,  7,  0,  0,  0,  0],
        [ 1,  1,  7,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  2,  3,  4,  5, 14,  7,  0,  0],
        [ 1,  1, 12,  9,  4,  5

In [33]:
# onto defining the network!
import torch.nn as nn

class NLM(nn.Module):
    # two things that you need to do
    # 1. init function (initializes all the **parameters** of the network)
    # 2. forward function (defines the forward propagation computations)

    def __init__(self, d_embedding, d_hidden, window_size, len_vocab):
        super(NLM, self).__init__() # init the base Module class
        self.d_emb = d_embedding
        self.embeddings = nn.Embedding(len_vocab, d_embedding)

        # concatenated embeddings > hidden
        self.W_hid = nn.Linear(d_embedding*window_size,  d_hidden)

        # hidden > output probability distribution over vocab
        self.W_out = nn.Linear(d_hidden, len_vocab)

    def forward(self, input): # each input will be a batch of prefixes (in this case 4)
        batch_size, window_size = input.size()
        embs = self.embeddings(input) # 4 x 2 x 5
        # print('embedding size:', embs.size())

        # next,  we want to concatenate the prefix embeddings together
        concat_embs = embs.view(batch_size, window_size * self.d_emb)
        # print('concatenated embs size:', concat_embs.size())

        # now, we project this to the hidden space
        hiddens = self.W_hid(concat_embs)
        # print('hidden size:', hiddens.size())

        # finally, project hiddens to vocabulary space
        outs = self.W_out(hiddens)
        # print('output size:', outs.size())

        return outs # return unnormalized probability, also known as **logits**


network = NLM(d_embedding=5, d_hidden=12, window_size=2, len_vocab=len(vocab))

num_epochs = 100 # how many times we are going to go through our entire training dataset
learning_rate = 0.1 # we can modify this if training isn't converging, etc.
loss_fn = nn.CrossEntropyLoss() # average cross entropy loss over each prefix in batch

# we will use vanilla gradient descent, you can experiment with others like Adam
optimizer = torch.optim.SGD(params=network.parameters(), lr=learning_rate)

# training loop
for i in range(num_epochs):
    logits = network(prefixes)
    loss = loss_fn(logits, labels)
    # now let's update our params to make the loss smaller
    # step 1: compute gradient (partial derivs of loss WRT parameters)
    loss.backward()

    # step 2: update params using gradient descent
    optimizer.step()

    # step 3: zero out the gradients for the next epoch
    optimizer.zero_grad()
    print('epoch: %d, loss: %0.2f' % (i, loss))




RuntimeError: mat1 and mat2 shapes cannot be multiplied (6x50 and 10x12)